In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import asyncio
import csv
from enum import Enum

from loguru import logger
from pydantic import BaseModel

from taskllm.optimizer.data import DataSet, Row
from taskllm.optimizer.methods import BanditTrainer, BayesianTrainer
from taskllm.optimizer.prompt.meta import PromptMode

# logger.remove()  # remove the old handler. Else, the old one will work (and continue printing DEBUG logs) along with the new handler added below'
# logger.add(sys.stdout, level="TRACE")  # add a new handler which has INFO as the default


class Ratings(Enum):
    ONE = "1"
    TWO = "2"
    THREE = "3"
    FOUR = "4"
    FIVE = "5"
    NA = "N/A"


class StarbucksReviewRating(BaseModel):
    rating: Ratings


def sentiment_scoring_function(
    row: Row[StarbucksReviewRating], output: StarbucksReviewRating | None
) -> float:
    if output is None:
        return -10
    if not row.expected_output:
        return 0
    logger.trace(f"Expected: {row.expected_output.rating}, Output: {output.rating}")
    if row.expected_output.rating == output.rating:
        return 1

    return 0


def load_file_as_dataset(path: str) -> DataSet:
    rows = []
    with open(path, "r", encoding="utf-8") as f:
        reader = csv.DictReader(f)
        for row in reader:
            rating = StarbucksReviewRating(rating=Ratings(row["Rating"]))
            rows.append(
                Row.create(
                    input_dictionary={
                        "review": row["Review"],
                        "name": row["name"],
                        "location": row["location"],
                        "date": row["Date"],
                    },
                    output=rating,
                )
            )
    return DataSet(rows=rows[:50], name="starbucks_reviews")



/Users/bill/dev/taskllm/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
csv_path = "./starbucks_reviews.csv"
dataset = load_file_as_dataset(csv_path)
trainer = BayesianTrainer(
    all_rows=dataset,
    task_guidance="determine the rating of this review",
    keys=["review", "name", "location", "date"],
    expected_output_type=StarbucksReviewRating,
    scoring_function=sentiment_scoring_function,
    num_iterations=2,  # Start with fewer iterations for testing
    candidates_per_iteration=2,  # Start with fewer candidates for testing
    prompt_mode=PromptMode.SIMPLE,
    models=[
        "anthropic/claude-3-haiku-20240307",
        "openai/gpt-4.1-nano-2025-04-14",
        "openai/gpt-4.1-mini-2025-04-14",
        "groq/gemma2-9b-it",
        "groq/qwen-qwq-32b"
    ],
)

2025-05-12 20:42:15.960 | INFO     | taskllm.optimizer.methods.bayesian:__init__:170 - Using CPU for Pyro/Torch computations
2025-05-12 20:42:15.960 | INFO     | taskllm.optimizer.methods.base:__init__:236 - All rows: 50
2025-05-12 20:42:15.960 | DEBUG    | taskllm.optimizer.methods.bayesian:__init__:683 - BayesianTrainer initialized


In [4]:
await trainer.train()

2025-05-12 20:25:44.708 | INFO     | taskllm.optimizer.methods.bayesian:train:687 - Starting Bayesian optimization with Pyro: 2 iterations, 2 candidates/iter.
2025-05-12 20:25:44.709 | INFO     | taskllm.optimizer.methods.bayesian:train:693 - Phase 1: Generating and evaluating initial candidates...
2025-05-12 20:25:44.709 | INFO     | taskllm.optimizer.prompt.meta:generate_spec:294 - Generating prompt content for: determine the rating of this review

Use plain language in the prompt you write
2025-05-12 20:25:44.709 | INFO     | taskllm.optimizer.prompt.meta:generate_spec:294 - Generating prompt content for: determine the rating of this review

Make the prompt you write as simple as possible
2025-05-12 20:25:59.605 | INFO     | taskllm.optimizer.methods.bayesian:train:715 - Generated 2 initial candidate prompts.
2025-05-12 20:25:59.607 | INFO     | taskllm.optimizer.methods.base:run_for_prompt:274 - Running for prompt 1. Read the provided review carefully to understand the overall sent


Give Feedback / Get Help: https://github.com/BerriAI/litellm/issues/new
LiteLLM.Info: If you need to debug this error, use `litellm._turn_on_debug()'.


Provider List: https://docs.litellm.ai/docs/providers



2025-05-12 20:27:55.657 | INFO     | taskllm.optimizer.methods.bayesian:train:727 - Fitting initial surrogate model
2025-05-12 20:27:55.659 | SUCCESS  | taskllm.optimizer.methods.bayesian:fit_surrogate_model:229 - Extracted features for 2 prompts
2025-05-12 20:27:55.660 | DEBUG    | taskllm.optimizer.methods.base:get_outputs:49 - Getting outputs for 40 rows using anthropic/claude-3-haiku-20240307
2025-05-12 20:27:55.661 | DEBUG    | taskllm.optimizer.methods.base:get_outputs:49 - Getting outputs for 40 rows using groq/qwen-qwq-32b
2025-05-12 20:27:55.709 | DEBUG    | taskllm.optimizer.methods.base:get_scores:66 - Calculated 40 scores
2025-05-12 20:27:55.710 | INFO     | taskllm.optimizer.methods.base:calculate_scores:91 - Model anthropic/claude-3-haiku-20240307 with prompt achieved score: 19.0000, Correct: 19, Incorrect: 21, Unlabelled: 0 out of 40
2025-05-12 20:27:55.710 | DEBUG    | taskllm.optimizer.methods.base:get_scores:66 - Calculated 40 scores
2025-05-12 20:27:55.710 | INFO    


Give Feedback / Get Help: https://github.com/BerriAI/litellm/issues/new
LiteLLM.Info: If you need to debug this error, use `litellm._turn_on_debug()'.


Provider List: https://docs.litellm.ai/docs/providers


Give Feedback / Get Help: https://github.com/BerriAI/litellm/issues/new
LiteLLM.Info: If you need to debug this error, use `litellm._turn_on_debug()'.


Provider List: https://docs.litellm.ai/docs/providers


Give Feedback / Get Help: https://github.com/BerriAI/litellm/issues/new
LiteLLM.Info: If you need to debug this error, use `litellm._turn_on_debug()'.


Provider List: https://docs.litellm.ai/docs/providers


Give Feedback / Get Help: https://github.com/BerriAI/litellm/issues/new
LiteLLM.Info: If you need to debug this error, use `litellm._turn_on_debug()'.


Provider List: https://docs.litellm.ai/docs/providers



2025-05-12 20:28:42.985 | INFO     | taskllm.optimizer.methods.bayesian:train:774 - Prompt content: 1. Read the review carefully.  
2. Determine the review's tone: look for words indicating positive, ...
2025-05-12 20:28:42.986 | INFO     | taskllm.optimizer.methods.bayesian:train:774 - Prompt content: 1. Carefully analyze the supplied review text.  
2. Assess the predominant sentiment of the review b...
2025-05-12 20:28:42.987 | INFO     | taskllm.optimizer.methods.bayesian:train:779 - Updating surrogate model with new data
2025-05-12 20:28:42.989 | SUCCESS  | taskllm.optimizer.methods.bayesian:fit_surrogate_model:229 - Extracted features for 4 prompts
2025-05-12 20:28:42.990 | DEBUG    | taskllm.optimizer.methods.base:get_outputs:49 - Getting outputs for 40 rows using anthropic/claude-3-haiku-20240307
2025-05-12 20:28:42.991 | DEBUG    | taskllm.optimizer.methods.base:get_outputs:49 - Getting outputs for 40 rows using groq/qwen-qwq-32b
2025-05-12 20:28:42.992 | DEBUG    | taskllm.opt


Give Feedback / Get Help: https://github.com/BerriAI/litellm/issues/new
LiteLLM.Info: If you need to debug this error, use `litellm._turn_on_debug()'.


Provider List: https://docs.litellm.ai/docs/providers



2025-05-12 20:29:13.024 | DEBUG    | taskllm.optimizer.methods.base:get_scores:66 - Calculated 10 scores
2025-05-12 20:29:13.025 | INFO     | taskllm.optimizer.methods.base:calculate_scores:91 - Model ModelsEnum.LLAMA_3_8B with prompt achieved score: 3.0000, Correct: 3, Incorrect: 7, Unlabelled: 0 out of 10
2025-05-12 20:29:13.026 | DEBUG    | taskllm.optimizer.methods.base:get_scores:66 - Calculated 10 scores
2025-05-12 20:29:13.027 | INFO     | taskllm.optimizer.methods.base:calculate_scores:91 - Model ModelsEnum.LLAMA_3_8B with prompt achieved score: 5.0000, Correct: 5, Incorrect: 5, Unlabelled: 0 out of 10
2025-05-12 20:29:13.029 | DEBUG    | taskllm.optimizer.methods.base:get_outputs:49 - Getting outputs for 10 rows using anthropic/claude-3-haiku-20240307
2025-05-12 20:29:13.046 | DEBUG    | taskllm.optimizer.methods.base:get_scores:66 - Calculated 10 scores
2025-05-12 20:29:13.046 | INFO     | taskllm.optimizer.methods.base:calculate_scores:91 - Model anthropic/claude-3-haiku-202